In [201]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re


julia_roberts_wiki_url = 'https://en.wikipedia.org/wiki/Julia_Roberts_filmography'
page = urlopen(julia_roberts_wiki_url)
soup = BeautifulSoup(page)
table=soup.find('table', class_='wikitable plainrowheaders sortable')
base_url = 'https://en.wikipedia.org/'


years=[]
names=[]
countries=[]

# get all available links for all movie pages
movies_url=[]
for row in table.findAll("tr"):
    headers=row.findAll('th')
    if(len(headers)==1):
        link = headers[0].find(href=True)
        if(link is not None):
            movies_url.append(base_url + link.get('href'))
            
            
def getSoupFromUrl(url):
    page = urlopen(url)
    return BeautifulSoup(page)

def getYearOfBirth(born_details):
    dataRegex = re.compile(r'\d\d\d\d')
    for text in born_details:
        match = dataRegex.search(text)
        if match is not None:
          group = match.group()
          return group
        
    return 'Year not found'
        
def getPlaceOfBirth(row):
    div = row.find('div', class_='birthplace')
    if div is not None:
        return div.find(text=True)
    else:
        #msometimes there is only link to the country
        country = row.find(href=True)
        if(country is not None):
            return country.find(text=True)
        else:
            return "Country Not Found"
        
def setArraysWithActorsData(actor_url):
    soup = getSoupFromUrl(actor_url)
    table = soup.find('table', class_='infobox')
    if table is None:
        print('cant find infobox table on ' + actor_url)
    else:
        #find only the name of the actor
        first_row = table.findAll("tr")[0]
        header = first_row.findAll('th')
        name = header[0].find(text=True)
        if name is None:
            name='Name not found'

        #find born details
        for row in table.findAll("tr"):
            if(row.find(text=True)=='Born'):
                born_details =row.findAll('td')[0].findAll(text=True)
                year = getYearOfBirth(born_details)
                country = getPlaceOfBirth(row)
                break;

        years.append(year)
        countries.append(country)
        names.append(name)

def extract_data_from_ul(ul):
    actor_urls=[]
    for li in ul.findAll('li'):
        actor_url = li.find(href=True)
        if actor_url is not None :
            actor_urls.append(base_url + actor_url.get('href'))
    for actor_url in actor_urls:
        setArraysWithActorsData(actor_url)
        
        
def extract_data_from_table(table):
    actor_urls=[]
    for row in table.findAll("tr"):
        cells=row.findAll('td')
        if(len(cells)==3 or len(cells)==2):
            actor_url = cells[0].find(href=True)
            if actor_url is not None:
                actor_urls.append(base_url + actor_url.get('href'))
        
    if len(actor_urls)==0:
       print('cells lenght problem')

    for actor_url in actor_urls:
        setArraysWithActorsData(actor_url)

    

for url in movies_url:
    print('===== ' + url)
    actor_urls=[]
    soup = getSoupFromUrl(url)
    cast_span_tag = soup.find('span', class_='mw-headline', text=re.compile('(cast)|(Cast)'))
    header_tag= cast_span_tag.parent
    actors = header_tag.findNext(['ul','table'])
    if(actors.name=='ul'):
        extract_data_from_ul(actors)
    else:
        extract_data_from_table(actors)
    df=pd.DataFrame(years,columns=['Year'])
    df['Name']=names
    df['Country']=countries

    df
  

    
    

    

    
    
    
    
    
 

   





===== https://en.wikipedia.org//wiki/Firehouse_(1987_film)
===== https://en.wikipedia.org//wiki/Satisfaction_(film)
===== https://en.wikipedia.org//wiki/Mystic_Pizza
cant find infobox table on https://en.wikipedia.org//wiki/Suzanne_Shepherd
===== https://en.wikipedia.org//wiki/Blood_Red_(film)
cant find infobox table on https://en.wikipedia.org//wiki/Joseph_Runningfox
===== https://en.wikipedia.org//wiki/Steel_Magnolias
===== https://en.wikipedia.org//wiki/Pretty_Woman
===== https://en.wikipedia.org//wiki/Flatliners
===== https://en.wikipedia.org//wiki/Sleeping_with_the_Enemy
===== https://en.wikipedia.org//wiki/Dying_Young
===== https://en.wikipedia.org//wiki/Hook_(film)
===== https://en.wikipedia.org//wiki/The_Player_(1992_film)
===== https://en.wikipedia.org//wiki/The_Pelican_Brief_(film)
===== https://en.wikipedia.org//wiki/I_Love_Trouble_(1994_film)
===== https://en.wikipedia.org//wiki/Pr%C3%AAt-%C3%A0-Porter_(film)
===== https://en.wikipedia.org//wiki/Something_to_Talk_About_(fil

HTTPError: HTTP Error 404: Not Found